In [2]:
from dataset.datasets import VideoClsDataset

/opt/conda/envs/videomae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
video_path="/tsukimi/datasets/Chiba/cut_videos_2label"
# mapping_file="/tsukimi/datasets/Chiba/persistence/video_label_map_filtered.json"
folder_map="/tsukimi/datasets/Chiba/persistence/video_folder_map.pkl"
import json
import os
import os.path as op
# with open(mapping_file) as f:
#     label_map = json.load(f)

import pickle
with open(folder_map, 'rb') as f:
    folder_map = pickle.load(f)
    
# video path like /tsukimi/datasets/Chiba/cut_videos_2label/6_S4_3T_RBG15_center.mov/restrainer_interaction-running-30.26-31.17.mov
path_label_list=[]    

for root, dirs, files in os.walk(video_path):
    for file in files:
        if file.endswith(".mov"):
            # parse the label
            splits=file.split('-')
            label0=splits[0]
            label1=splits[1]
            start=float(splits[2])
            end=float(splits[3][:-4])
            path_label_list.append((op.join(root, file), (label0, label1), (start, end)))

In [3]:
print(path_label_list[0])

('/tsukimi/datasets/Chiba/cut_videos_2label/8_S2_4T_RBG3_center.mov/running-running-141.09-142.0.mov', ('running', 'running'), (141.09, 142.0))


In [20]:
# map label set to 0~n
label_map={}
for i, label in enumerate(label_set):
    label_map[label]=i

for i, path_label in enumerate(path_label_list):
    path, label=path_label.split()
    label=label_map[label]
    path_label_list[i]=' '.join([path, str(label)])

In [1]:
# # save path_label_list to csv file
# import pandas as pd
# df=pd.DataFrame(path_label_list)
# df

# train validation test split
from sklearn.model_selection import train_test_split
import pandas as pd
train, temp=train_test_split(path_label_list, test_size=0.3, random_state=42)
val, test=train_test_split(temp, test_size=0.5, random_state=42)

print(len(train),len(val),len(test))

# check label distribution
label_dict={}
for filepath, label, _ in train:
    for l in label:
        label_dict[l]=label_dict.get(l, 0)+1
print(label_dict)

# save to csv
# import pandas as pd
df_train=pd.DataFrame(train)
df_val=pd.DataFrame(val)
df_test=pd.DataFrame(test)

def encode_label(df):
    df[1]=df[1].map(lambda labels:'&'.join(labels))
    df.drop(columns=[2], inplace=True)
    return df

df_train=encode_label(df_train)
df_val=encode_label(df_val)
df_test=encode_label(df_test)
save_path="/tsukimi/datasets/Chiba/baseline"
df_train.to_csv(op.join(save_path, 'train.csv'), index=False, header=False)
df_val.to_csv(op.join(save_path, 'val.csv'), index=False, header=False)
df_test.to_csv(op.join(save_path, 'test.csv'), index=False, header=False)

NameError: name 'path_label_list' is not defined

In [28]:
# read train csv
import pandas as pd
df=pd.read_csv(op.join(save_path, 'train.csv'), header=None)
df.head()

,0,1
0,/tsukimi/datasets/Chiba/cut_videos_2label/1_S3...,restrainer_interaction&running
1,/tsukimi/datasets/Chiba/cut_videos_2label/3_S4...,unsupported_rearing&idle_actions
2,/tsukimi/datasets/Chiba/cut_videos_2label/0_S3...,interaction_with_partner&running
3,/tsukimi/datasets/Chiba/cut_videos_2label/1_S1...,restrainer_interaction&running
4,/tsukimi/datasets/Chiba/cut_videos_2label/6_S3...,climbing_on_side&unsupported_rearing


In [30]:
import torch
import torch.nn.functional as F

# Example logits and target
x = torch.tensor([[0.1, 2.2, 3.1], [1.0, 2.0, 1.5]])  # logits for 2 samples, 3 classes each
target = torch.tensor([2, 1])  # true class indices

# Compute log_softmax
logprobs = F.log_softmax(x, dim=-1)
print(logprobs)
# Gather the log probabilities of the actual classes
selected_log_probs = logprobs.gather(dim=-1, index=target.unsqueeze(1))

# Resulting tensor of selected log probabilities
selected_log_probs

tensor([[-3.3759, -1.2759, -0.3759],
        [-1.6803, -0.6803, -1.1803]])


tensor([[-0.3759],
        [-0.6803]])

In [13]:
import torch
import torch.nn as nn

# Assuming the following mapping of labels to indices: A -> 0, B -> 1, C -> 2
label_mapping = {'A': 0, 'B': 1, 'C': 2}

# Example raw labels
raw_labels = ['A&B', 'B&C', 'A&C','A&A']

# Function to convert raw labels to one-hot encoded format
def ground_truth_decoder(labels,num_classes=len(label_mapping)):
    decoded = torch.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        parts = label.split('&')
        for part in parts:
            decoded[i, label_mapping[part]] += 1
    return decoded/2

# One-hot encoded targets
targets = ground_truth_decoder(raw_labels)
print(targets)
# Example outputs from the model (logits)
outputs = torch.randn(4, 3)  # Batch size of 3, and 3 classes

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Calculate loss
loss = criterion(outputs, targets)
print("Loss:", loss.item())


tensor([[0.5000, 0.5000, 0.0000],
        [0.0000, 0.5000, 0.5000],
        [0.5000, 0.0000, 0.5000],
        [1.0000, 0.0000, 0.0000]])
Loss: 0.8683392405509949


In [4]:
from eventutils import multi_label_accuracy,custom_multi_label_pred,ground_truth_decoder
import torch
# Example outputs from the model (logits)
outputs = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]])
# Example targets
targets = torch.tensor([[0, 0.5, 0.5], [0.5, 0, 0.5], [0, 0, 1]])
preds=custom_multi_label_pred(outputs)
print(preds)
acc=multi_label_accuracy(outputs, targets)
print(acc)

tensor([[0.0000, 0.5000, 0.5000],
        [0.0000, 0.5000, 0.5000],
        [0.0000, 0.0000, 1.0000]])
tensor(0.8333)


In [ ]:
import torch
ground_truth_labels=[torch.Tensor([0, 1, 1]), torch.Tensor([1, 0, 1])]
pred_labels=[torch.Tensor([1, 1, 1]), torch.Tensor([1, 0, 1])]
ground_truth_labels=torch.cat(ground_truth_labels)
pred_labels=torch.cat(pred_labels)
confusion_matrix=multi_label_confusion_matrix(ground_truth_labels,pred_labels)
confusion_matrix

In [1]:
import torch

def multi_label_confusion_matrix(ground_truth_labels, pred_labels):
    """
    Calculate confusion matrix for multi-label classification.
    Args:
    - ground_truth_labels (torch.Tensor): ground truth binary labels where labels are considered present if non-zero.
    - pred_labels (torch.Tensor): predicted binary labels, same shape as ground_truth_labels.
    
    Returns:
    - torch.Tensor: Confusion matrix of shape (4, C) where C is the number of classes.
      The rows represent TP, FP, FN, TN for each class.
    """

    # Convert probabilities to binary labels for confusion matrix calculation
    ground_truth_labels = (ground_truth_labels > 0).float()
    pred_labels = (pred_labels > 0).float()
    
    # Number of classes
    num_classes = ground_truth_labels.shape[1]

    # Initialize the confusion matrix components
    TP = torch.zeros(num_classes)
    FP = torch.zeros(num_classes)
    FN = torch.zeros(num_classes)
    TN = torch.zeros(num_classes)

    # Calculate TP, FP, FN, TN for each class
    for i in range(num_classes):
        TP[i] = (ground_truth_labels[:, i] * pred_labels[:, i]).sum()
        FP[i] = ((1 - ground_truth_labels[:, i]) * pred_labels[:, i]).sum()
        FN[i] = (ground_truth_labels[:, i] * (1 - pred_labels[:, i])).sum()
        TN[i] = ((1 - ground_truth_labels[:, i]) * (1 - pred_labels[:, i])).sum()

    # Combine into a single confusion matrix
    confusion_matrix = torch.stack((TP, FP, FN, TN))

    return confusion_matrix

# Example usage
ground_truth_labels = torch.tensor([[0.5, 0.5, 0], [0, 0, 1], [0.5, 0, 0.5]])
pred_labels = torch.tensor([[1, 0, 0], [0, 1, 1], [0, 0, 1]])

# Calculate the confusion matrix
conf_matrix = multi_label_confusion_matrix(ground_truth_labels, pred_labels)
print("Confusion Matrix (TP, FP, FN, TN):")
print(conf_matrix)


Confusion Matrix (TP, FP, FN, TN):
tensor([[1., 0., 2.],
        [0., 1., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])


/opt/conda/envs/videomae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path="/tsukimi/datasets/Chiba/baseline/datalist/"
# read val.csv
import pandas as pd
import os.path as op
df=pd.read_csv(op.join(path, 'val.csv'), header=None)
df

,0,1
0,/tsukimi/datasets/Chiba/cut_videos_2label/2_S2...,restrainer_interaction&immobility
1,/tsukimi/datasets/Chiba/cut_videos_2label/1_S2...,climbing_on_side&immobility
2,/tsukimi/datasets/Chiba/cut_videos_2label/9_S4...,immobility&running
3,/tsukimi/datasets/Chiba/cut_videos_2label/2_S2...,running&immobility
4,/tsukimi/datasets/Chiba/cut_videos_2label/9_S1...,running&immobility
...,...,...
275,/tsukimi/datasets/Chiba/cut_videos_2label/1_S3...,interaction_with_partner&interaction_with_partner
276,/tsukimi/datasets/Chiba/cut_videos_2label/8_S4...,interaction_with_partner&interaction_with_partner
277,/tsukimi/datasets/Chiba/cut_videos_2label/8_S3...,interaction_with_partner&interaction_with_partner
278,/tsukimi/datasets/Chiba/cut_videos_2label/0_S1...,interaction_with_partner&interaction_with_partner


In [ ]:
ground_truth_labels = torch.tensor([[0.5, 0.5, 0], [0, 0, 1], [0.5, 0, 0.5]])
